In [8]:
from spacerocks.observing import Observatory
from spacerocks.time import Time
from spacerocks import SpaceRock

from spacerocks.spice import SpiceKernel
kernel = SpiceKernel()
kernel.load("/Users/kjnapier/data/spice/latest_leapseconds.tls")
kernel.load("/Users/kjnapier/data/spice/de440s.bsp")

import numpy as np

origin = "SSB"
reference_plane = "J2000"


import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from spacerocks.nbody import Simulation, Integrator

In [9]:
w84 = Observatory.from_obscode('w84')

In [3]:
epoch = Time.now()

planets_names = ["sun", 
                 "jupiter barycenter", 
                 "saturn barycenter", 
                 "uranus barycenter", 
                 "neptune barycenter"]
rocks = [SpaceRock.from_horizons("Arrokoth", epoch=Time.now(), origin="ssb", reference_plane="ECLIPJ2000")]
planets = [SpaceRock.from_spice(name, epoch, reference_plane="ECLIPJ2000", origin='ssb') for name in planets_names]

sim = Simulation()

sim.set_epoch(epoch)
sim.set_reference_plane("ECLIPJ2000")
sim.set_origin('ssb')
sim.set_integrator(Integrator.ias15(timestep=20.0))


for planet in planets:
    sim.add(planet)
    
for rock in rocks:
    sim.add(rock)

sim.move_to_center_of_mass()


In [4]:
observations = []    
for idx in range(0, 600, 5):
    sim.integrate(epoch + idx)
    observer = w84.at(epoch + idx, reference_plane=reference_plane, origin=origin)
    rock = sim.get_particle("Arrokoth")
    obs = rock.observe(observer)
    observations.append(obs)

In [6]:
from spacerocks.orbfit import gauss

In [7]:
fit_rocks = gauss(o1=observations[0], o2=observations[20], o3=observations[10], min_distance=0)
for fit_rock in fit_rocks:
    print(fit_rock.a(), fit_rock.e(), fit_rock.inc(), fit_rock.node(), fit_rock.arg())

44.24274893908718 0.03805024901657919 0.3694607460720733 0.0423376885627325 5.943054132486585
1.2865625154352125 0.9824010766347244 0.4885234296054679 0.40868978386620913 1.004391954557038
1.38106056841974 0.36551942155291056 0.4053335290085945 0.01789838317402071 2.795058658866398
